# Deploy Python on Firebase Hosting with Cloud Run - Firecasts Tutorial
You will be using Firebase Hosting and Cloud Run to host a Flask server. Firebase Hosting can support dynamic server code like Python. You can do that with Cloud Run. 

Cloud Run is a way to serverlessly run containers. If it can fit into a stateless Docker container, you can hook that up with Firebase Hosting, and you get all the benefits of Firebase Hosting CDN.

## Things to know before you get started

### Firebase Hosting Project Creation
You will obviously need to create a **Firebase Hosting** account to start your first project. Log in with your preferred email at https://firebase.google.com.

At the following prompt, select **Get Started**,

![Firebase1](imgs/firebase1.png)

Click **Create a project**,

![Firebase2](imgs/firebase2.png)

Start with a name for your project. (*Tip: Set your project name and project ID to the same thing*)

![Firebase3](imgs/firebase3.png)

#### Firebase Hosting Pricing Plans
Something to note before getting started is that **Firebase Hosting** and **Cloud Run** integration does not operate within the free Spark plan. You need to upgrade to the **Blaze plan**, which requires a credit card.

![Plans](imgs/plan2.png)

However, we will be using **Cloud Build** and **Cloud Run**, which do have their own free tiers. So, even if you enter a credit card you can still operate within the free tier without any charges.

![Google](imgs/googlecloud.png)

### Cloud SDK
To get started, we need to use the **Google Cloud SDK**, which you can install. Install by following either tutorial,

- [Install the Google Cloud CDI](https://cloud.google.com/sdk/docs/install-sdk)
- [Install Google Cloud SDK & CLI for Mac & Linux Users](https://www.youtube.com/watch?v=k-8qFh8EfFA)

To begin,

## 1. Set up project directory
Set up project directory with the following structure,

    /project  

        /project/server  
            
            /project/server/Docker

            /project/server/src

                /project/server/src/app.py 
                
        /project/static  
        
        /project/.firebaserc  
        
        /project/firebase.json

## 2. Create `/project/server/src/app.py` 

`app.py`
    
    from flask import Flask
    import os

    app = Flask(__name__)

    @app.route('/')
    def hello_world():
        return 'Hello World'

    if __name__ == '__main__':
        app.run(debug=True, host='0.0.0.0', port=int(os.environ.get('PORT', 8080)))

## 3. Create `/project/server/Docker`

`Docker`

    FROM python:3.7

    RUN pip install Flask gunicorn

    COPY src/ app/
    WORKDIR /app

    ENV PORT 8080

    CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 app:app

## 4. Login with `gcloud auth login`
Once you have set up the **Google Cloud SDK**, you will need to log in by running the following command in your project directory,

![Login](imgs/login.png)

It will prompt you to select an account to continue using **Google Cloud SDK**,

![Login2](imgs/login2.png)

If you were successful, you should be taken to this screen,

![Success](imgs/success.png)


## 5. Run `gcloud init`

After you log in, initialize with `gcloud init`

![init](imgs/init.png)

Select **[2] Create a new configuration** when prompted and enter a name for it.

![init2](imgs/init2.png)

Specify the account you are using for this configuration.

![init3](imgs/init3.png)

And finally, select your cloud project.

![init4](imgs/init4.png)

## 6. Build a container and store it in Google Container Registry.
The next step is to build a container and store it in Google Container Registry, that way we can deploy it to Cloud Run. To build it, you'll need Docker locally. But if you don't have Docker locally, or don't want to install it, you can actually do it through **Cloud Build**.

First, you need to `cd` into the folder with the `Dockerfile`.

![cd](imgs/cd.png)

Then call,

    gcloud builds submit --tag gcr.io/<project-id>/<container-name>

Specifying your **project ID** and granting the container a **name**,

![cd2](imgs/cd2.png)

It is going to ask you if you need to enable some APIs, enter yes.

![cd3](imgs/cd3.png)

## 7. Deploy to Cloud Run
Now, we can deploy to Cloud Run with the command `gcloud run deploy --image gcr.io/<project-id>/<container-name>`, giving it the image which is the same as the tag.

![deploy](imgs/deploy.png)

Enable the API,

![deploy](imgs/deploy2.png)

For Firebase Hosting, we need to make sure it is in `us-central1`,

![deploy](imgs/deploy3.png)

Allow unauthenticated invocations because it is a web server,

![deploy](imgs/deploy4.png)

That is going to kick off the deploy. Once it's done, it is going to give you a link.

![deploy](imgs/deploy5.png)

Paste the link into the browser. And you can see we have our **Hello World** page. Now we have our serverless container up and running. Let's hook it up to Firebase Hosting.

![deploy](imgs/deploy6.png)

## 8. Hook Up Cloud Run to Firebase Hosting
Hooking up Cloud Run to Firebase Hosting means we need to install the **Firebase CLI**. First, `cd` to your project directory,

![hook](imgs/hook.png)

Run `npm init -y`,

![hook](imgs/hook2.png)

Now you can install the **Firebase CLI** with `npm i -D firebase-tools`

![hook](imgs/hook3.png)

## 9. Create `/project/.firebaserc` and `/project/firebase.json`
Create a `.firebaserc` and `firebase.json` in your project directory. You can do this with one command, `node_modules/.bin/firebase init hosting`, but let's create it from scratch.

Create the following `.firebaserc`, replacing `default` with your project ID.

`.firebaserc`

    {
    "projects": {
        "default": "tutorial-firebase-helloworld"
    }
}

Create the following, `firebase.json` replacing service ID with the one you specified in your tag,

`firebase.json`

    {
        "hosting": {
            "public": "static",
            "ignore": [
                "firebase.json",
                "**/.*",
                "**/node_modules/**"
            ],
            "rewrites": [{
                "source": "**",
                "run": {
                    "serviceId": "flask-fire"
                }
            }]
        }
    }

## 9. Confirm everything is working
To see if everything is working, run `node_modules/.bin/firebase serve`

![confirm](imgs/confirm.png)

Once that kicks off, it will run on `http://localhost:5000`.

![confirm](imgs/confirm2.png)

Copy and paste into a browser,

![confirm](imgs/confirm3.png)

## 10. Deploy to Cloud Run
Kill the program with CTRL+C and `cd` to **/project/server**.

![run2](imgs/run2.png)

Then deploy to Cloud Run with `gcloud run deploy <container-name> --region us-central1 --platform managed --image gcr.io/<project-id>/<container-name>`

![run3](imgs/run3.png)

Once it finishes deploying, you can `cd` back up to your project directory so you can deploy to Firebase Hosting with `node_modules/.bin/firebase deploy --only hosting`,

![run4](imgs/run4.png)

You use the tag `--only hosting` because our Cloud Run container has been deployed. So we just need to deploy with Firebase Hosting.

Now that it is done, we have our **hosting URL**!

![url](imgs/url.png)